In [2]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [7]:
class Dictionary(object):
    def __init__(self) -> None:
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word) -> None:
        if word not in self.word2idx:        
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self) -> int:
        return len(self.word2idx)

In [15]:
class TextProcess(object):

    def __init__(self) -> None:
        self.dictionary = Dictionary()
    
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        rep_tensor = torch.LongTensor(tokens) # tensor of length tokens
        index = 0

        # Capture the indices of each word in the tensor for the entire story
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        
        # Divide the data into batches based on batch size 
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches*batch_size]
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor


In [19]:
embed_size = 128         # Input features to LSTM. idx -> word embedding size
hidden_size = 1024       # Num of hidden neurons or num of hidden units
num_layers = 1           # 1 layer of LSTM. If 2, 2 LSTM layers stacked on top of each other
num_epochs = 20
batch_size = 20
timesteps = 30           # we look at 30 previous words to predict next word
learning_rate = 0.002    # we can use LR scheduler here

In [20]:
corpus = TextProcess()
rep_tensor = corpus.get_data('17.alice.txt', batch_size)
print(rep_tensor.shape)

torch.Size([20, 1484])


In [27]:
vocab_size = len(corpus.dictionary)
print(vocab)

5290


In [22]:
# batches based on time steps
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [31]:
class TextGenerator(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) # performs the mapping from idx to word embedding
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h) # x -> input, h -> initialzed hidden state and memory state
        # Reshape out(batch_size, timesteps, hidden_size) -> (batch_size*timesteps, hidden_size) to be appropriate for FC layer
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [32]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [34]:
# Train the model
for epoch in range(num_epochs):

    # set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        # Get mini batch inputs and targets
        inputs = rep_tensor[:, i:i+timesteps]
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]

        outputs, _ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1)) # targets must be in 1 dimension from documentation

        model.zero_grad()
        loss.backward()

        clip_grad_norm(model.parameters(), 0.5) # to prevent exploding gradient problem. 0.5 is max allowed value.
        optimizer.step()

        step = (i+1) // timesteps
        if step % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")

/tmp/ipykernel_29101/962583981.py:18: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  clip_grad_norm(model.parameters(), 0.5) # to prevent exploding gradient problem. 0.5 is max allowed value.


Epoch [1/20], Loss: 8.5805
Epoch [2/20], Loss: 5.9532
Epoch [3/20], Loss: 5.2731
Epoch [4/20], Loss: 4.7043
Epoch [5/20], Loss: 4.2109
Epoch [6/20], Loss: 3.7910
Epoch [7/20], Loss: 3.3719
Epoch [8/20], Loss: 2.8796
Epoch [9/20], Loss: 2.4959
Epoch [10/20], Loss: 2.2119
Epoch [11/20], Loss: 1.8760
Epoch [12/20], Loss: 1.6148
Epoch [13/20], Loss: 1.3648
Epoch [14/20], Loss: 1.0963
Epoch [15/20], Loss: 0.8079
Epoch [16/20], Loss: 0.4968
Epoch [17/20], Loss: 0.3395
Epoch [18/20], Loss: 0.2142
Epoch [19/20], Loss: 0.1404
Epoch [20/20], Loss: 0.1007


In [35]:
# Test the model - Generate text and see
with torch.no_grad():
    with open('17.results.txt', 'w') as f:
        state = (torch.zeros(num_layers, 1, hidden_size),
              torch.zeros(num_layers, 1, hidden_size))

        input = torch.randint(0, vocab_size, (1,)).long().unsqueeze(1)
    
        for i in range(500):
            output, _ = model(input, state)
            print(output.shape)

            # Sample a word if from the exponential of the output
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            print(word_id)

            # Replace input with sampled word id for the next time step
            input.fill_(word_id)

            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print(f"Sample [{i+1}/500] words and save to results.txt")

torch.Size([1, 5290])
1047
torch.Size([1, 5290])
960
torch.Size([1, 5290])
9
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3161
torch.Size([1, 5290])
3209
torch.Size([1, 5290])
76
torch.Size([1, 5290])
27
torch.Size([1, 5290])
362
torch.Size([1, 5290])
389
torch.Size([1, 5290])
110
torch.Size([1, 5290])
2423
torch.Size([1, 5290])
5
torch.Size([1, 5290])
439
torch.Size([1, 5290])
209
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3779
torch.Size([1, 5290])
98
torch.Size([1, 5290])
80
torch.Size([1, 5290])
3
torch.Size([1, 5290])
1617
torch.Size([1, 5290])
5
torch.Size([1, 5290])
1237
torch.Size([1, 5290])
20
torch.Size([1, 5290])
6
torch.Size([1, 5290])
333
torch.Size([1, 5290])
5
torch.Size([1, 5290])
3832
torch.Size([1, 5290])
1857
torch.Size([1, 5290])
5
torch.Size([1, 5290])
27
torch.Size([1, 5290])
28
torch.Size([1, 5290])
35
torch.Size([1, 5290])
2678
torch.Size([1, 5290])
1249
torch.Size([1, 5290])
265
torch.Size([1, 5290])
1909
torch.Size([1, 5290])
203
torch.Size([1, 5290])
3546